In [3]:
import pandas as pd
import json

In [6]:
with open('./GPTFramesUnique.json', 'r') as f:
    data = json.load(f)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(data.items()), columns=['File Name', 'Content'])

In [8]:
len(df)

2036

In [9]:
df = df[df['Content'] != "Code NA"]

In [10]:
len(df)

1555

In [20]:
list(df[100:200]["Content"])[:5]

['```python\n# Let\'s now split up the data into train and validation sets\nn = int(0.9*len(data))  # first 90% will be train, rest val\ntrain_data = data[:n]\nval_data = data[n:]\n\nblock_size = 8\ntrain_data[:block_size+1]\n\nx = train_data[:block_size]\ny = train_data[1:block_size+1]\nfor t in range(block_size):\n    context = x[t:t+1]\n    target = y[t]\n    print(f"when input is {context} the target: {target}")\n```',
 '```python\nn = int(0.9*len(data))  # first 90% will be train, rest val\ntrain_data = data[:n]\nval_data = data[n:]\n\nblock_size = 8\ntrain_data[:block_size+1]\n\nx = train_data[:block_size]\ny = train_data[1:block_size+1]\nfor t in range(block_size):\n    context = x[t:t+1]\n    target = y[t]\n    print(f"when input is {context} the target: {target}")\n```',
 '```python\nn = int(0.9 * len(data))\ntrain_data = data[:n]\nval_data = data[n:]\n\nblock_size = 8\ntrain_data[:block_size+1]\n\nx = train_data[:block_size]\ny = train_data[1:block_size+1]\nfor t in range(blo

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

In [ ]:
merge_snippets_prompt = """
I will provide you a list of code snippets that may contain overlapping or redundant elements. 
Your task is to sequentially process these snippets and merge them into a single, coherent code block. 
While doing so, ensure that redundancies are eliminated, and overlapping sections are appropriately merged. 
Do not add any additional code beyond what is provided; simply integrate the snippets to create a unified and functional result.
Do not include any comments that say stuff like here is the extracted code, etc. 
It is important that you do not omit any code as well.
Just return the final merged code block. 

Here is the list of code snippets in order: {}
"""

def process_one_batch(client, code_snippets):
    prompt = merge_snippets_prompt.format(code_snippets)
    messages = [{"role": "user", "content": prompt}]

    response_params = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "temperature": 0,
    }

    response = client.chat.completions.create(**response_params)
    consolidated_code = response.choices[0].message.content
    return consolidated_code

In [94]:
# lis100 = list(df[:100]["Content"])
# out100 = process_one_batch(client, lis100)
# print(out100)

In [95]:
# lis100 = list(df[:100]["Content"])
# out100 = process_one_batch(client, lis100)
# print(out100)

In [96]:
# chunk_size = 100
# code_snippet_list = []
# for start in range(0, len(df), chunk_size):
#     chunk = list(df[start:start + chunk_size]["Content"])
#     merged_code_block = process_one_batch(client, chunk)
#     code_snippet_list.append(merged_code_block)

# # final_code = process_one_batch(client, code_snippet_list)

In [ ]:
# len(code_snippet_list)

8

In [ ]:
# parallel code to process batches

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Function to process one batch, wrapped for parallel execution
def process_chunk(index, start, chunk_size, df, client):
    chunk = list(df[start:start + chunk_size]["Content"])
    merged_code_block = process_one_batch(client, chunk)
    return index, merged_code_block

# Parallel execution using ThreadPoolExecutor with tqdm
chunk_size = 200
num_workers = 3
code_snippet_list = [None] * ((len(df) + chunk_size - 1) // chunk_size)  # Preallocate list

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    # Creating a list of futures with indices
    futures = [
        executor.submit(process_chunk, idx, start, chunk_size, df, client)
        for idx, start in enumerate(range(0, len(df), chunk_size))
    ]
    
    # Use tqdm to track the progress of completed futures
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing chunks"):
        index, result = future.result()
        code_snippet_list[index] = result

Processing chunks: 100%|██████████| 8/8 [01:02<00:00,  7.78s/it]


In [ ]:
# Solution 1 for merging

# final_code_prompt = """
# I will provide you a list of code snippets that may contain overlapping or redundant elements. 
# Your task is to sequentially process these snippets and concatenate them. 
# While doing so, ensure that redundancies are eliminated, and overlapping sections are appropriately merged. 
# You should not re-order these snippets.
# Do not modify or add any additional code beyond what is provided.
# It is important that you do not omit any code as well.
# Just return the final code without redundant snippets.

# Here is the list of code snippets in order: {}
# """

# def get_final_code(client, code_snippets):
#     prompt = final_code_prompt.format(code_snippets)
#     messages = [{"role": "user", "content": prompt}]

#     response_params = {
#         "model": "gpt-4o",
#         "messages": messages,
#         "temperature": 0,
#     }

#     response = client.chat.completions.create(**response_params)
#     consolidated_code = response.choices[0].message.content
#     return consolidated_code

abcde


In [ ]:
# solution 2 for merging
# more structured divide and conquer approach that iteratively merges adjacent strings
# like (1, 2) (3, 4) etc and then again

merge_prompt = """
Concatenate these two code snippets. If there is any redundancy or overlap, merge that part accordingly.
It is very important that you do not re-order the given snippets, you need to merge them in the order given. 
You only need to merge them when you find a strong overlap in some part. Otherwise, you need to mostly concatenate them. 
It is also extremely important that you do not omit any code.
Return the final full code. 

Code snippet 1: 
{}

Code snippet 2: 
{}
"""

def get_merged_code(client, code_snippet1, code_snippet2):
    prompt = merge_prompt.format(code_snippet1, code_snippet2)
    messages = [{"role": "user", "content": prompt}]

    response_params = {
        "model": "gpt-4o",
        "messages": messages,
        "temperature": 0,
    }

    response = client.chat.completions.create(**response_params)
    merged = response.choices[0].message.content
    return merged

def merge_strings(strings):
    merged_list = []
    for i in range(0, len(strings) - 1, 2):
        merged_list.append(get_merged_code(client, strings[i], strings[i + 1]))
    if len(strings) % 2 != 0:  # Add the last string if the list is odd
        merged_list.append(strings[-1])
    return merged_list

def merge_until_one_string(strings):
    count = 1
    while len(strings) > 1:
        print(f"Iteration {count}: Combining {len(strings)} strings")
        count += 1
        strings = merge_strings(strings)
    return strings[0] if strings else ""

final_string = merge_until_one_string(code_snippet_list)

Iteration 1: Combining 8 strings
Iteration 2: Combining 4 strings
Iteration 3: Combining 2 strings


In [153]:
with open("new.txt", "w") as f:
  f.write(final_string)